## Import

In [1]:
!pip install rasterio
!pip install rioxarray
!pip install geopandas
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00


In [2]:
import xarray as xr
import pandas as pd
import rasterio
from rasterio.mask import mask
from rasterio.transform import from_bounds
import rioxarray
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from glob import glob

## Init

Your data repository should look like **this** after downloading all input data (non-exhaustive list of files):

![image.png](attachment:image.png)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
main_data_folder = "/content/drive/MyDrive/data/"

# Administration geometries from GADM

[GADM](https://gadm.org/) provides maps and spatial data for all countries and their sub-divisions. Here is a light version limited to European countries.

In [ ]:
gadm_filepath = main_data_folder + "gadm_410_europe.gpkg"

## Parametrize

In [ ]:
country_code = "NET" # Germany
cityname = "Utrecht"

## Manipulate and vizualize

In [ ]:
gadm_gdf = gpd.read_file(gadm_filepath)

print(f"Number of rows in GADM: {len(gadm_gdf)}")
gadm_gdf.head()

In [ ]:
filtered_gadm_gdf = gadm_gdf[(gadm_gdf.GID_0 == country_code) & (gadm_gdf.NAME_2 == cityname)]

print(f"Number of rows in GADM after filtering: {len(filtered_gadm_gdf)}")
filtered_gadm_gdf.head()

In [ ]:
one_city_gdf = filtered_gadm_gdf.dissolve()

one_city_gdf.head()

In [ ]:
one_city_gdf.plot()

Warning! Depending on the country you're studying, the administrative level of the city might be different.

For example, "Berlin" in Germany requires the `gadm_gdf.NAME_2 == cityname` condition (ie. admin level 2), however "Lille" in France requires the `gadm_gdf.NAME_5 == cityname` condition (ie. admin level 5).

It can even vary within a country if a city is divided in boroughs (ex: "Paris" in France requires the `gadm_gdf.NAME_2 == cityname` condition (ie. admin level 2)).

# ERA5-Land Meteorological data (frequency = daily)

ERA5-Land is a reanalysis dataset providing a consistent view of the evolution of land variables over several decades at an enhanced resolution compared to ERA5.

The data, processed on a daily temporal resolution, originally stems from a public hourly dataset: [ERA5-Land hourly data from 1950 to present](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=overview) on the [Climate Data Store](https://cds.climate.copernicus.eu/).

The available land variables are:
- Daily maximum of hourly [2 metre temperature](https://apps.ecmwf.int/codes/grib/param-db/167) in Kelvin,
- Daily mean of hourly [total precipitation](https://apps.ecmwf.int/codes/grib/param-db/228) in meters,
- Daily mean of hourly [10 metre U wind component](https://apps.ecmwf.int/codes/grib/param-db/165) in meters per second,
- Daily mean of hourly [10 metre V wind component](https://apps.ecmwf.int/codes/grib/param-db/166) in meters per second.

In [ ]:
era5_data_folder = main_data_folder + "derived-era5-land-daily-statistics/"

variable2statistic = {
    "2m_temperature": "daily_maximum",
    "total_precipitation": "daily_mean",
    "10m_u_component_of_wind": "daily_mean",
    "10m_v_component_of_wind": "daily_mean",
}

variable2datavar = {
    "2m_temperature": "t2m",
    "total_precipitation": "tp",
    "10m_u_component_of_wind": "u10",
    "10m_v_component_of_wind": "v10",
}

## Parametrize

In [ ]:
variable = "10m_u_component_of_wind"
year = 2020

In [ ]:
statistic = variable2statistic[variable]
datavar = variable2datavar[variable]

## Manipulate and vizualize

Please open the [xarray User Guide](https://docs.xarray.dev/en/stable/user-guide/index.html) to see how to manipulate the data (especially in the "Core operations" chapter).

Here are a few examples of operations:

### One dataset

In [ ]:
filepath = f"{era5_data_folder}{year}_{variable}_{statistic}.nc"

In [ ]:
ds = xr.open_dataset(filepath)
ds

In [ ]:
lat, lon = 48.8566, 2.3522

ds[datavar].sel(latitude=lat, longitude=lon, method="nearest").plot()

In [ ]:
day = pd.to_datetime(f"{year}-06-01")

ds[datavar].sel(valid_time=day).plot()

### Multiple datasets

In [ ]:
filepaths = glob(f"{era5_data_folder}*_{variable}_{statistic}.nc")

In [ ]:
ds = xr.open_mfdataset(filepaths, combine="by_coords")
ds

In [ ]:
ds[datavar].sel(latitude=lat, longitude=lon, method="nearest").plot()

# Normalized Difference Vegetation Index from Sentinel-2 (frequency = every 3 months)

The Normalized difference vegetation index (known as NDVI) is a simple, but effective index for quantifying green vegetation. Its values range from -1 to 1. More details on [sentinelhub](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ndvi/).

In [ ]:
ndvi_data_folder = main_data_folder + "sentinel2_ndvi/"

def quarter2timeperiod(year, quarter):
    if quarter == 1:
        return f"{year-1}-12-01_{year}-03-01"
    elif quarter == 2:
        return f"{year}-03-01_{year}-06-01"
    elif quarter == 3:
        return f"{year}-06-01_{year}-09-01"
    elif quarter == 4:
        return f"{year}-09-01_{year}-12-01"
    else:
        raise ValueError("quarter must be in 1, 2, 3, 4")

## Parametrize

In [ ]:
available_years = [2020, 2021, 2022, 2023]
available_quarters = [1, 2, 3, 4]

In [ ]:
year = 2020
quarter = 4

In [ ]:
timeperiod = quarter2timeperiod(year, quarter)

filepath  = f"{ndvi_data_folder}ndvi_{timeperiod}.tif"

## Manipulate and vizualize

In [ ]:
def convert_ndvi_to_real_scale(ndvi_img, out_meta):
    # The NDVI is stored in int8 format on a 0/254 scale, and nodata is 255.
    # This function converts it to a float format on a -1/1 scale, and replace nodata with np.nan.
    ndvi_img = ndvi_img.astype(float)
    ndvi_img[ndvi_img == out_meta["nodata"]] = np.nan
    ndvi_img = ndvi_img / 254 * 2 - 1
    return ndvi_img

def get_out_image_and_metadata(filepath, one_city_gdf):
    with rasterio.open(filepath) as src:
        one_city_gdf_in_good_crs = one_city_gdf.to_crs(src.crs)
        city_geometry = [one_city_gdf_in_good_crs.geometry.iloc[0]]
        out_image, out_transform = mask(src, city_geometry, crop=False)
        out_meta = src.meta

        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        real_out_image = convert_ndvi_to_real_scale(out_image, out_meta)

    return real_out_image, out_meta

In [ ]:
real_out_image, out_meta = get_out_image_and_metadata(filepath, one_city_gdf)

In [ ]:
plt.figure()
plt.imshow(real_out_image[0], cmap='Greens', vmin=-1, vmax=1)
plt.colorbar(label="NDVI")
plt.title(f"NDVI over {cityname} during {timeperiod}")
plt.axis('off')
plt.show()

In [ ]:
mean_ndvi = np.nanmean(real_out_image[0])
print(f"Mean NDVI over {cityname} during {timeperiod}: {mean_ndvi:.3f}")

In [ ]:
mean_ndvi_time_series = pd.Series(dtype=float)
for year in available_years:
    for quarter in available_quarters:
        timeperiod = quarter2timeperiod(year, quarter)
        filepath  = f"{ndvi_data_folder}ndvi_{timeperiod}.tif"
        date = pd.to_datetime(timeperiod.split("_")[1])
        current_real_out_image, _ = get_out_image_and_metadata(filepath, one_city_gdf)
        current_mean_ndvi = np.nanmean(current_real_out_image[0])
        mean_ndvi_time_series.loc[date] = current_mean_ndvi

mean_ndvi_time_series = mean_ndvi_time_series.sort_index()

plt.figure()
mean_ndvi_time_series.plot(color='green')
plt.title(f"Mean NDVI over {cityname} time series")
plt.ylabel("Mean NDVI")
plt.xlabel("Time")
plt.ylim(-1, 1)
plt.grid()
plt.show()

## Reproject ERA5-Land features on the NDVI grid

In [ ]:
da = ds[datavar].sel(valid_time=day)
da

In [ ]:
origin_transform = from_bounds(
    ds.longitude.min().item(),
    ds.latitude.min().item(),
    ds.longitude.max().item(),
    ds.latitude.max().item(),
    len(ds.longitude),
    len(ds.latitude)
)
origin_crs = "EPSG:4326"

target_transform = out_meta["transform"]
target_crs = out_meta["crs"]

In [ ]:
reprojected_da = da.rio.write_crs(origin_crs).rio.reproject(
    dst_crs=target_crs,
    shape=(out_meta["height"], out_meta["width"]),
    transform=target_transform,
)

In [ ]:
reprojected_da.plot()